In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.2 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 9.9 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 1.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 18.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from scipy.stats import rankdata
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import optuna 

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/PS-S3/Ep7/train.csv'
file_key_2 = 'Tabular-Playground-Series/PS-S3/Ep7/test.csv'
file_key_3 = 'Tabular-Playground-Series/PS-S3/Ep7/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

In [3]:
train_lgb = train.copy()
test_lgb = test.copy()

X = train_lgb.drop(columns = ['id', 'booking_status'], axis = 1)
Y = train_lgb['booking_status']

test_lgb = test_lgb.drop(columns = ['id'], axis = 1)

# Optuna

In [11]:
class Objective:

    def __init__(self, seed):
        # Hold this implementation specific arguments as the fields of the class.
        self.seed = seed

    def __call__(self, trial):
        ## Parameters to be evaluated
        param = dict(objective = 'binary',
                     metric = 'auc',
                     tree_method = 'gbdt', 
                     n_estimators = trial.suggest_int('n_estimators', 300, 10000),
                     learning_rate = trial.suggest_float('learning_rate', 0.001, 1, log = True),
                     max_depth = trial.suggest_int('max_depth', 3, 12),
                     lambda_l1 = trial.suggest_float('lambda_l1', 0.01, 10.0, log = True),
                     lambda_l2 = trial.suggest_float('lambda_l2', 0.01, 10.0, log = True),
                     num_leaves = trial.suggest_int('num_leaves', 2, 100),
                     bagging_fraction = trial.suggest_float('bagging_fraction', 0.2, 0.9),
                     feature_fraction = trial.suggest_float('feature_fraction', 0.2, 0.9)
                    )

        scores = []
        
        skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = self.seed)
#         skf = KFold(n_splits = 5, shuffle = True, random_state = self.seed)

        for train_idx, valid_idx in skf.split(X, Y):

            X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
            Y_train , Y_valid = Y.iloc[train_idx] , Y.iloc[valid_idx]

            model = LGBMClassifier(**param).fit(X_train, Y_train)

#             preds_train = model.predict(X_train)
            preds_valid = model.predict_proba(X_valid)[:, 1]

            score = roc_auc_score(Y_valid, preds_valid)
            scores.append(score)

        return np.mean(scores)

In [ ]:
N_TRIALS = 50
SEED = 42

study = optuna.create_study(direction = 'maximize')
study.optimize(Objective(SEED), n_trials = N_TRIALS)

[I 2023-02-14 22:54:11,811] A new study created in memory with name: no-name-3857d91a-2a01-4455-b318-854b1394555e


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=2.802583529854927, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.802583529854927
[LightGBM] [Warning] feature_fraction is set=0.6612758006087783, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6612758006087783
[LightGBM] [Warning] lambda_l2 is set=0.2503162468006942, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.2503162468006942
[LightGBM] [Warning] bagging_fraction is set=0.4532478175495665, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4532478175495665
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=2.802583529854927, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.802583529854927
[LightGBM] [Warning] feature_fraction is set=0.6612758006087783, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6612758006087783
[LightGBM] [Warning] lambda_l2 is set=0.25031

[I 2023-02-14 22:54:41,958] Trial 0 finished with value: 0.8985081686963646 and parameters: {'n_estimators': 2133, 'learning_rate': 0.028105649406782333, 'max_depth': 7, 'lambda_l1': 2.802583529854927, 'lambda_l2': 0.2503162468006942, 'num_leaves': 59, 'bagging_fraction': 0.4532478175495665, 'feature_fraction': 0.6612758006087783}. Best is trial 0 with value: 0.8985081686963646.


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=6.519289996685612, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.519289996685612
[LightGBM] [Warning] feature_fraction is set=0.7313785907146209, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7313785907146209
[LightGBM] [Warning] lambda_l2 is set=0.24685228467051237, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.24685228467051237
[LightGBM] [Warning] bagging_fraction is set=0.489200377044646, subsample=1.0 will be ignored. Current value: bagging_fraction=0.489200377044646
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=6.519289996685612, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.519289996685612
[LightGBM] [Warning] feature_fraction is set=0.7313785907146209, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7313785907146209
[LightGBM] [Warning] lambda_l2 is set=0.24685

[I 2023-02-14 22:54:55,079] Trial 1 finished with value: 0.898705572547707 and parameters: {'n_estimators': 1277, 'learning_rate': 0.05751970146941916, 'max_depth': 7, 'lambda_l1': 6.519289996685612, 'lambda_l2': 0.24685228467051237, 'num_leaves': 70, 'bagging_fraction': 0.489200377044646, 'feature_fraction': 0.7313785907146209}. Best is trial 1 with value: 0.898705572547707.


...

[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.8000428960313068, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8000428960313068
[LightGBM] [Warning] feature_fraction is set=0.3276503087505557, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3276503087505557
[LightGBM] [Warning] lambda_l2 is set=0.9064502993804515, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.9064502993804515
[LightGBM] [Warning] bagging_fraction is set=0.5318905232258622, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5318905232258622
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.8000428960313068, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8000428960313068
[LightGBM] [Warning] feature_fraction is set=0.3276503087505557, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.3276503087505557
[LightGBM] [Warning] lambda_l2 is set=0.9

[I 2023-02-14 23:30:28,193] Trial 37 finished with value: 0.8977526229152041 and parameters: {'n_estimators': 3492, 'learning_rate': 0.004080354751364609, 'max_depth': 12, 'lambda_l1': 0.8000428960313068, 'lambda_l2': 0.9064502993804515, 'num_leaves': 31, 'bagging_fraction': 0.5318905232258622, 'feature_fraction': 0.3276503087505557}. Best is trial 18 with value: 0.9004299441816952.


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.41350658159277914, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.41350658159277914
[LightGBM] [Warning] feature_fraction is set=0.47754926985088797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47754926985088797
[LightGBM] [Warning] lambda_l2 is set=3.2867121081087, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.2867121081087
[LightGBM] [Warning] bagging_fraction is set=0.6114694295732271, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6114694295732271
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=0.41350658159277914, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.41350658159277914
[LightGBM] [Warning] feature_fraction is set=0.47754926985088797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.47754926985088797
[LightGBM] [Warning] lambda_l2 is set=3

[I 2023-02-14 23:32:00,099] Trial 38 finished with value: 0.8960320803873254 and parameters: {'n_estimators': 8490, 'learning_rate': 0.020151679772125047, 'max_depth': 8, 'lambda_l1': 0.41350658159277914, 'lambda_l2': 3.2867121081087, 'num_leaves': 37, 'bagging_fraction': 0.6114694295732271, 'feature_fraction': 0.47754926985088797}. Best is trial 18 with value: 0.9004299441816952.


[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=1.1928959892851738, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1928959892851738
[LightGBM] [Warning] feature_fraction is set=0.5159164857341607, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5159164857341607
[LightGBM] [Warning] lambda_l2 is set=1.9634643621536472, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.9634643621536472
[LightGBM] [Warning] bagging_fraction is set=0.5762378533905923, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5762378533905923
[LightGBM] [Warning] Unknown parameter: tree_method
[LightGBM] [Warning] lambda_l1 is set=1.1928959892851738, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1928959892851738
[LightGBM] [Warning] feature_fraction is set=0.5159164857341607, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5159164857341607
[LightGBM] [Warning] lambda_l2 is set=1.9

In [9]:
study.best_trial.values

[0.8990248712785431]

In [10]:
study.best_trial.params

{'n_estimators': 1636,
 'learning_rate': 0.022275330029586263,
 'max_depth': 8,
 'lambda_l1': 0.02226373938692656,
 'lambda_l2': 0.04119756851367091,
 'num_leaves': 64,
 'bagging_fraction': 0.494316389151127,
 'feature_fraction': 0.5364579362079585}